# EXTRACION DE CARACTERISTICAS DE AUDIO

Relacionando librerias

In [292]:
import librosa
from librosa import feature
import numpy as np
import pandas as pd
from glob import glob
import csv

Estableciendo directorios de trabajo

In [293]:
data_dir = 'C:\\Data\\2_Preprocesamiento\\Pruebas_5\\0_Sin_clasificar'
raiz = 'C:\\Data\\2_Preprocesamiento\\Pruebas_5\\' 
audio_files = glob(data_dir + '/*.wav')
txt_files = glob(data_dir + '/*.txt')
#print(audio_files)

Funciones y arreglos

In [294]:
fn_list_i = [
 feature.chroma_stft,
 feature.spectral_centroid,
 feature.spectral_bandwidth,
 feature.spectral_rolloff,
 feature.mfcc,
 feature.melspectrogram
]
 
fn_list_ii = [
 feature.rmse,
 feature.zero_crossing_rate
]

header =[
 'Audio',
 'especie',
 'chroma_stft',
 'spectral_centroid',
 'spectral_bandwidth',
 'spectral_rolloff',
 'mfcc',
 'melspectrogram',
 'rmse',
 'zero_crossing_rate',
 'pitch',
 'Corte',
 'startime',
 'finishtime',
 'id',
 'freqmin',
 'freqmax' 
]

def crear_archivo(ruta):
    archivo = open(ruta, 'w')
    return archivo
    
    
def leer_archivo (archivo):
    arch1 = open(archivo, 'r')
    contenido = arch1.read()
    return contenido
    
    
def escribir_archivo(archivo, texto):
    archivo.write(texto)

def get_feature_vector(file, etiqueta, y,sr): 
   feat_vect = [file]
   feat_vect_i = [ np.mean(funct(y,sr)) for funct in fn_list_i]
   feat_vect_ii = [ np.mean(funct(y)) for funct in fn_list_ii]
   feat_vect_iii = [(etiqueta)]
   feature_vector = feat_vect + feat_vect_iii + feat_vect_i + feat_vect_ii 
#print(feature_vector)
   return feature_vector

def get_pitches(y,sr):
    # Track frequencies from a real spectrogram
    pitches, magnitudes = librosa.piptrack(y, sr)
    # Select out pitches with high energy
    pitches = pitches[magnitudes > np.median(magnitudes)]
    pitch = librosa.pitch_tuning(pitches)
    return pitch

def get_features_texto(file_txt):
    cont = leer_archivo(file_txt)
    #cont = cont.replace("'","")
    lista = cont.split(",") 
    datos = [str(lista[0])] + [float(lista[1])] + [float(lista[2])] + [str(lista[3])] + [float(lista[4])] + [float(lista[5])]
    #print(datos)
    return datos

Generando Vector de Trabajo

In [295]:
audios_feat = []
for file in audio_files:
   y, sr = librosa.load(file,sr=None)
   #print(file)
   filename = file.replace(data_dir + '\\', '')
   etiqueta = file.replace(raiz, '')
   etiqueta = etiqueta.replace('\\'+filename, '')
   file_txt = file[0:len(file)-4]+'.txt'
   #print(file_txt)
   text_vector = get_features_texto(file_txt)
   feature_vector = get_feature_vector(filename, etiqueta, y, sr)
   pitch_vector = get_pitches(y,sr)
   audios_feat.append(feature_vector + [pitch_vector]+ text_vector) 

Definiendo DATAFRAME y archivo CSV

In [296]:
norm_output = raiz+'df_'+etiqueta+'.csv'
norm_output = norm_output.replace('//', '/')
df = pd.DataFrame(audios_feat)
df.columns = header 
tiempo=df['finishtime']-df['startime']
df=df.assign(duration=tiempo)
del df['startime']
del df['finishtime']
df.to_csv(norm_output)